In [1]:
import functools
import itertools
import warnings
warnings.filterwarnings('ignore')
import sys
sys.tracebacklimit=0
# package(s) related to time, space and id
import logging
import datetime, time
import platform

# you need these dependencies (you can get these from anaconda)
# package(s) related to the simulation
import simpy
import pandas as pd

# spatial libraries 
import pyproj
import shapely.geometry
from simplekml import Kml, Style

# package(s) for data handling
import numpy as np
import scipy.optimize
import matplotlib.pyplot as plt
from matplotlib.ticker import StrMethodFormatter
import tqdm

# OpenTNSim
import opentnsim

# Used for mathematical functions
import math             

# Used for making the graph to visualize our problem
import networkx as nx   

import plotly.express as px
from plotly.subplots import make_subplots

#logging.basicConfig(level=logging.DEBUG) #turn on all the debug messages
#logging.basicConfig(level=logging.INFO)# turn off all the debug messages

logger = logging.getLogger('notebook')



In [2]:
# Make your preferred class out of available mix-ins.
TransportResource = type(
    "Vessel",
    (
        opentnsim.core.Identifiable,
        opentnsim.core.Movable,
        opentnsim.core.Routeable,
        opentnsim.core.VesselProperties,  # needed to add vessel properties
        opentnsim.core.HasContainer,  # needed to calculate filling degree for draught
        opentnsim.core.ConsumesEnergy,
        opentnsim.core.ExtraMetadata,
    ),
    {},
)  

In [3]:
# Create a dict with all important settings
data_vessel = {"env": None,
               "name": None,
               "route": None,
               "geometry": None,
               "v": None,  # m/s
               "type": 'Dortmund-Eems (L <= 74 m)', # <-- note that inputs from hereon are due to the added mixins!
               "B": 11.4,
               "L": 110,
               "capacity": 2500,   # maximum designed payload
               "level":2500,     # actual payload
               "H_e": None, 
               "H_f": None, 
               "T": None,
               "P_installed": 1750.0,   
               "P_tot_given": None, # kW
               "L_w": 3.0 ,
               "C_B":0.85, 
               "C_year":1990,
               "current_year":None
              }   
vessel = TransportResource(**data_vessel)

In [4]:
def T2Payload(self, T_strategy, vesl_type):
#def calculate_actual_T_and_payload(self, h_min, vesl_type="Dry_SH"):
    """ Calculate payload based on Van Dorsser et al
    (https://www.researchgate.net/publication/344340126_The_effect_of_low_water_on_loading_capacity_of_inland_ships)

    Given a route we get several possible draught (T_strategy) with veolicity (v_computed) combinations for the moving vessel, this step is done via T2v simulation.
    At this step, we calculate the payload for the possible draughts (T_strategy).

    Input:
    - T_strategy
    - vessel types: "Container","Dry_SH","Dry_DH","Barge","Tanker"

    Output:
    - corresponding payload for the T_strategy for different vessel types     

    """
    #Design draft T_design, refer to Table 5

    Tdesign_coefs = dict({"intercept":0,
                     "c1": 1.7244153371,
                     "c2": 2.2767179246,
                     "c3": 1.3365379898,
                     "c4": -5.9459308905,
                     "c5": 6.2902305560*10**-2,
                     "c6": 7.7398861528*10**-5,
                     "c7": 9.0052384439*10**-3,
                     "c8": 2.8438560877
                     })

    assert vesl_type in ["Container","Dry_SH","Dry_DH","Barge","Tanker"],'Invalid value vesl_type, should be "Container","Dry_SH","Dry_DH","Barge" or "Tanker"'
    if vesl_type == "Container":
        [dum_container, dum_dry,
        dum_barge, dum_tanker] = [1,0,0,0]
    elif vesl_type == "Dry_SH":
        [dum_container, dum_dry,
        dum_barge, dum_tanker] = [0,1,0,0]
    elif vesl_type == "Dry_DH":
        [dum_container, dum_dry,
        dum_barge, dum_tanker] = [0,1,0,0]
    elif vesl_type == "Barge":
        [dum_container, dum_dry,
        dum_barge, dum_tanker] = [0,0,1,0]
    elif vesl_type == "Tanker":
        [dum_container, dum_dry,
        dum_barge, dum_tanker] = [0,0,0,1]

    T_design = Tdesign_coefs['intercept'] + (dum_container * Tdesign_coefs['c1']) + \
                                            (dum_dry * Tdesign_coefs['c2']) + \
                                            (dum_barge * Tdesign_coefs['c3']) +\
                                            (dum_tanker * Tdesign_coefs['c4']) +\
                                            (Tdesign_coefs['c5'] * dum_container * self.L**0.4 * self.B**0.6) +\
                                            (Tdesign_coefs['c6'] * dum_dry * self.L**0.7 * self.B**2.6)+\
                                            (Tdesign_coefs['c7'] * dum_barge * self.L**0.3 * self.B**1.8) +\
                                            (Tdesign_coefs['c8'] * dum_tanker * self.L**0.1 * self.B**0.3)

    #Empty draft T_empty, refer to Table 4
    
    Tempty_coefs = dict({"intercept": 7.5740820927*10**-2,
                "c1": 1.1615080992*10**-1,
                "c2": 1.6865973494*10**-2,
                "c3": -2.7490565381*10**-2,
                "c4": -5.1501240744*10**-5,
                "c5": 1.0257551153*10**-1,
                "c6": 2.4299435211*10**-1,
                "c7": -2.1354295627*10**-1,
                })


    if vesl_type == "Container":
        [dum_container, dum_dry,
        dum_barge, dum_tanker] = [1,0,0,0]
    elif vesl_type == "Dry_SH":
        [dum_container, dum_dry,
        dum_barge, dum_tanker] = [0,0,0,0]
    elif vesl_type == "Dry_DH":
        [dum_container, dum_dry,
        dum_barge, dum_tanker] = [0,1,0,0]
    elif vesl_type == "Barge":
        [dum_container, dum_dry,
        dum_barge, dum_tanker] = [0,0,1,0]

    elif vesl_type == "Tanker":
        [dum_container, dum_dry,
        dum_barge, dum_tanker] = [0,0,0,1]

    # dum_container and dum_dry use the same "c5"
    T_empty = Tempty_coefs['intercept']  + (Tempty_coefs['c1'] * self.B) + \
                                           (Tempty_coefs['c2'] * ((self.L * T_design) / self.B)) + \
                                           (Tempty_coefs['c3'] * (np.sqrt(self.L * self.B)))  + \
                                           (Tempty_coefs['c4'] * (self.L * self.B * T_design)) +  \
                                           (Tempty_coefs['c5'] * dum_container) + \
                                           (Tempty_coefs['c5'] * dum_dry)   + \
                                           (Tempty_coefs['c6'] * dum_tanker) + \
                                           (Tempty_coefs['c7'] * dum_barge)



    #Capacity indexes, refer to Table 3 and eq 2
    CI_coefs = dict({"intercept": 2.0323139721 * 10**1,

            "c1": -7.8577991460 * 10**1,
            "c2": -7.0671612519 * 10**0,
            "c3": 2.7744056480 * 10**1,
            "c4": 7.5588609922 * 10**-1,
            "c5": 3.6591813315 * 10**1
            })
    # Capindex_1 related to actual draft (especially used for shallow water)
    Capindex_1 = CI_coefs["intercept"] + (CI_coefs["c1"] * T_empty) + (CI_coefs["c2"] * T_empty**2)  +  (
    CI_coefs["c3"] * T_strategy) + (CI_coefs["c4"] * T_strategy**2)   + ( CI_coefs["c5"] * (T_empty * T_strategy))
    # Capindex_2 related to design draft
    Capindex_2 = CI_coefs["intercept"] + (CI_coefs["c1"] * T_empty) + (CI_coefs["c2"] * T_empty**2)   + (
    CI_coefs["c3"] * T_design) + (CI_coefs["c4"] * T_design**2)  + (CI_coefs["c5"] * (T_empty * T_design))

    #DWT design capacity, refer to Table 6 and eq 3
    capacity_coefs = dict({"intercept": -1.6687441313*10**1,
         "c1": 9.7404521380*10**-1,
         "c2": -1.1068568208,
         })

    DWT_design = capacity_coefs['intercept'] + (capacity_coefs['c1'] * self.L * self.B * T_design) + (
     capacity_coefs['c2'] * self.L * self.B * T_empty) # designed DWT
    DWT_actual = (Capindex_1/Capindex_2)*DWT_design # actual DWT of shallow water


    if T_strategy < T_design:
        consumables=0.04 #consumables represents the persentage of fuel weight,which is 4-6% of designed DWT
                          # 4% for shallow water (Van Dosser  et al. Chapter 8,pp.68).

    else:
        consumables=0.06 #consumables represents the persentage of fuel weight,which is 4-6% of designed DWT
                          # 6% for deep water (Van Dosser et al. Chapter 8, pp.68).

    fuel_weight=DWT_design*consumables #(Van Dosser et al. Chapter 8, pp.68).
    Payload_strategy = DWT_actual-fuel_weight # payload=DWT-fuel_weight


    return  Payload_strategy

In [5]:
T_strategy = [2.15, 2.10, 2.05, 2.00, 1.95, 1.90, 1.85]

In [6]:
# prepare the work to be done
# create a list of all combinations
work3 = list(itertools.product(T_strategy))

# prepare a list of dictionaries for pandas
rows3 = []
for item in work3:
    row3 = {"T_strategy": item[0]}
    rows3.append(row3)

    # these are all the simulations that we want to run
# convert them to dataframe, so that we can apply a function and monitor progress
work3_df = pd.DataFrame(rows3)
work3_df.head(8)

,T_strategy
0,2.15
1,2.10
2,2.05
3,2.00
4,1.95
5,1.90
6,1.85


In [13]:
Strategies = []

for i, row3 in tqdm.tqdm(work3_df.iterrows()):
    T_strategy = row3['T_strategy']
    
    Payload_strategy = T2Payload(vessel, T_strategy, vesl_type = "Dry_SH")
    
    Strategy ={}
    Strategy.update(row3)
    Strategy['Payload_strategy'] = Payload_strategy
    Strategies.append(Strategy)
    

7it [00:00, 6700.17it/s]


In [20]:
Strategies_df = pd.DataFrame(Strategies)
# Strategies_df.head(16)
with pd.option_context('display.float_format', '{:.2f}'.format):
    display(Strategies_df)

,T_strategy,Payload_strategy
0,2.15,1458.35
1,2.10,1399.62
2,2.05,1340.97
3,2.00,1282.39
4,1.95,1223.88
5,1.90,1165.45
6,1.85,1107.10


In [ ]:
Payload_strategy = T2Payload(vessel, T_strategy, vesl_type = "Dry_SH")
print(Payload_strategy)

In [ ]:
# unfinished

def Payload2T(vessel, T_strategy, bounds=(0, 10)):
    """Compute vessel velocity given the minimum water depth and possible actual draught 
    
    bounds is the limits where to look for a solution for the velocity [m/s]
    returns velocity [m/s]
    """
    
    def seek_T_given_Payload(v, vessel):
        """function to optimize"""

        
        # compute the maximum draught a vessel in stationary can have in the minimum water depth section, 
        # the ukc is static ukc
        h_min = vessel.h_min
        vessel.v = v
        
        Payload_strategy = vessel.calculate_strategy_payload(T_strategy, vesl_type) 
       
        # compute difference between given draught (T_strategy) and computed draught (T_computed)
        T_strategy = vessel._T  # the user provided T
        
        diff = T_strategy - T_calculated
        
       
        return diff ** 2        
        

    # fill in some of the parameters that we already know
    fun = functools.partial(seek_payload_given_T, vessel=vessel, h_min = vessel.h_min)       

    # lookup a minimum
    fit = scipy.optimize.minimize_scalar(fun, bounds=bounds, method='bounded')

    # check if we found a minimum
    if not fit.success:
        raise ValueError(fit)    

    v_comupted =  fit.x

#    print('v_comupted: {:.2f} m/s'.format(v_comupted))

    return v_comupted
    
